# Search Postal Address Code - DAEV

#### This notebook search for teh corrects address by consuming the post office API.

In [16]:
# Let´s import the libraries
import pandas as pd
import numpy as np

# Library to work with exceptions
import logging

In [17]:
# Let´s import ours functions
%run generalFunctions.ipynb

In [18]:
# let´s configurate the data frame display properties
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [19]:
# let´s create a dataFrame basead in csv file
df_address = pd.read_csv("dataIn/Layout_CEP_nova.csv", encoding = "ISO-8859-1", sep = ";", header = 0)

In [20]:
# Let´s transform the column cep in string
df_address['cep'] = df_address['cep'].astype(str)

In [21]:
# Let´s see the our dataframe shape
df_address.shape

(18, 7)

In [22]:
# Let´s see our datafrme
df_address["newCEP"] = ""
df_address["newLogradouro"] = ""
df_address["newComplemento"] = ""
df_address["newBairro"] = ""
df_address["newCidade"] = ""
df_address["newEstado"] = ""
df_address["newStatus"] = ""
df_address["newMessage"] = ""

In [23]:
# Let´s see the our dataframe shape
df_address.shape

(18, 15)

In [24]:
# Local variabels
pageRequest = ""
pageRequestGetAdress = ""
cpfCorrectShape = ""

# Let´s see the dataframe b
for i in range(len(df_address)): 
    
    try:
    
        # Let´s to set a dataFrame CPF in local variable
        cpfCorrectShape = fnc_AdjustCepBody(str(df_address["cep"][i])[:8])
        
    except Exception as e:
        
        # Let´s print the message error
        print(str(e))
        
    else:

        # Let´s check if some exeption will happen
        try:

            # Let´s search the CEP, consuming the API
            pageRequest = fnc_GetCEP(cpfCorrectShape)

        # Some exception happened
        except Exception as e:
            
            # let´s to insert the new values in the dataFrame
            df_address["newStatus"][i] == "ERRO API: " + str(e)

        # No exception happened
        else:

            # let´s validade the JSON returns
            if ("erro" in pageRequest):
                
                try:
                               
                    # Let´s try to search the CEP trough Address
                    pageRequestGetAdress = fnc_getCepByAddress(fnc_AdjustAddressString(str(df_address["logradouro"][i])))
                
                except Exception as e:
                
                    #let´s to insert the new values in the dataFrame
                    df_address["newStatus"][i] = ""
                    df_address["newMessage"][i] = "ERRO API: " + str(e)
                
                else:
                    
                    # let´s validade the JSON returns
                    if ("erro" in pageRequestGetAdress or len(pageRequestGetAdress) == 0):
                        
                        # let´s to insert the new values in the dataFrame
                        df_address["newStatus"][i] = ""
                        df_address["newMessage"][i] = "ERRO: CEP nao localizado, verifique o CPF da base."
                        
                    else:
                        
                        # Let´s check if more than one address was returnd
                        if len(pageRequestGetAdress) > 1:
                            
                            # let´s to insert the new values in the dataFrame
                            df_address["newStatus"][i] = "ATENCAO"
                            df_address["newMessage"][i] = "ATENCAO: Foram retornados varios enderecos."
                            
                        else:
                        
                            # let´s to insert the new values in the dataFrame
                            df_address["newCEP"][i] = fnc_AdjustCepBody(pageRequestGetAdress[0]["cep"])
                            df_address["newLogradouro"][i] = pageRequestGetAdress[0]["logradouro"]
                            df_address["newComplemento"][i] = pageRequestGetAdress[0]["complemento"]
                            df_address["newBairro"][i] = pageRequestGetAdress[0]["bairro"]
                            df_address["newCidade"][i] = pageRequestGetAdress[0]["localidade"]
                            df_address["newEstado"][i] = pageRequestGetAdress[0]["uf"]
                            df_address["newStatus"][i] = "SUCESSO:"
                            df_address["newMessage"][i] = "Sucesso: Dados localizados com sucesso."

            # Let´s to insert the CEPs information in our dataframe
            else:
                
                # let´s to insert the new values in the dataFrame
                df_address["newCEP"][i] = fnc_AdjustCepBody(pageRequest["cep"])
                df_address["newLogradouro"][i] = pageRequest["logradouro"]
                df_address["newComplemento"][i] = pageRequest["complemento"]
                df_address["newBairro"][i] = pageRequest["bairro"]
                df_address["newCidade"][i] = pageRequest["localidade"]
                df_address["newEstado"][i] = pageRequest["uf"]
                df_address["newStatus"][i] = "SUCESSO:"
                df_address["newMessage"][i] = "Sucesso: Dados localizados com sucesso."

<ipython-input-24-8df2be7a3e15>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_address["newCEP"][i] = fnc_AdjustCepBody(pageRequestGetAdress[0]["cep"])
<ipython-input-24-8df2be7a3e15>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_address["newLogradouro"][i] = pageRequestGetAdress[0]["logradouro"]
<ipython-input-24-8df2be7a3e15>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_address["newComplemento"][i] = pageRequestG

In [25]:
# Let´s export our dataFrame to a CSV file
msgInformation = fnc_exportsDfToCsv(df_address,"base_CEP_Tratada")

if msgInformation == "OK":
    print("Arquivo exportado com sucesso")
else:
    print("Erro: {error}".format(error = str(msgInformation)))

Arquivo exportado com sucesso
